In [3]:
import time
from decimal import Decimal
from hypothesis import given, strategies as st


# Motivation

1. Verify correctness
2. Verify correctness after modifications (regression testing)
3. Example-based, how to use documentation
4. Provoke good design: SOLID, flexible, modular

<img src="https://www.worldofagile.com/wp-content/uploads/2018/03/Agile-Test-Pyramid.png">

In [4]:
# Let's say we want to implement frange

In [4]:
def frange(start, stop=None, step=1):
    if stop is None:
        stop = start
        start = 0
#     if step == 0:
#         raise ValueError('0 cant be used as step')

    start = Decimal(str(start))
    stop = Decimal(str(stop))
    step = Decimal(str(step))
    result = start

    while (step > 0 and result < stop) or (step < 0 and result > stop):
        yield result
        result += step


In [5]:
# How to verify correctness?

for i in frange(0.1, 5, 0.1):
    print(i)

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0
1.1
1.2
1.3
1.4
1.5
1.6
1.7
1.8
1.9
2.0
2.1
2.2
2.3
2.4
2.5
2.6
2.7
2.8
2.9
3.0
3.1
3.2
3.3
3.4
3.5
3.6
3.7
3.8
3.9
4.0
4.1
4.2
4.3
4.4
4.5
4.6
4.7
4.8
4.9


In [8]:
# Primitive tests with asserts

assert list(frange(5)) == [0, 1, 2, 3, 4]
assert list(frange(2, 5)) == [2, 3, 4]
assert list(frange(2, 10, 2)) == [2, 4, 6, 8]
assert list(frange(10, 2, -2)) == [10, 8, 6, 4]
assert list(frange(2, 5.5, 1.5)) == [2, 3.5, 5]
assert list(frange(1, 5)) == [1, 2, 3, 4]
assert list(frange(0, 5)) == [0, 1, 2, 3, 4]
assert list(frange(1, 5, 0)) == []
assert list(frange(0, 0)) == []
assert list(frange(100, 0)) == []

print('SUCCESS')

SUCCESS


In [11]:
# Verbose error messages can be passed as a second param

assert list(frange(5)) == [0, 1, 2, 3, 6], f'{list(frange(5))} != [0, 1, 2, 3, 4, 6]'


# Is lazy evaluated on error, so inexpensive
assert list(frange(5)) == [0, 1, 2, 3, 4], [0]*10000000000000

AssertionError: [Decimal('0'), Decimal('1'), Decimal('2'), Decimal('3'), Decimal('4')] != [0, 1, 2, 3, 4, 6]

# Hypothesys

Good for computational test with ethalone to compare: (de)serializers, (de)encoders, sorting, etc

pip install hypothesis

https://hypothesis.readthedocs.io/en/latest/quickstart.html

In [12]:
@given(
    st.integers(min_value=-1000, max_value=1000),
    st.integers(min_value=-1000, max_value=1000),
    st.integers(min_value=-10, max_value=10).filter(lambda x: x != 0)
)
def test_frange(a, b, c):
    print(a, b, c)
    assert list(frange(a, b, c)) == list(range(a, b, c))

test_frange()

44 649 5
170 609 -3
-434 -87 -8
-474 679 5
607 226 -2
1000 966 -10
-725 503 9
-37 118 -2
-975 -500 -7
-264 471 -4
-264 -471 -4
-287 -769 3
-287 -287 3
-257 -257 -4
-799 -257 1
-257 -257 1
-586 486 -10
-859 -513 5
0 256 5
0 256 5
0 0 5
12 0 -7
0 0 -7
-829 -107 7
-107 -107 7
-107 -107 7
-107 -256 -5
-107 -256 -1
-256 -256 -1
-256 257 1
779 826 5
779 779 5
779 779 5
815 -768 -2
-768 -768 -2
-768 -768 -2
-768 -1 2
-776 158 8
158 158 8
158 158 8
0 0 -8
0 0 -8
0 0 1
0 0 1
-720 -711 -7
-720 -720 -7
-720 258 1
258 1 -8
258 1 8
258 258 8
257 -512 1
-378 530 -3
-378 -378 -3
-361 -513 -10
-513 -513 -10
-258 -258 -1
-258 -258 -1
-258 -257 -3
942 314 10
871 -512 10
871 258 4
871 871 4
871 871 4
-3 3 4
-3 -3 4
660 -271 -5
660 660 -5
660 513 -3
513 513 -3
513 513 -8
582 257 8
257 257 8
-308 1000 3
-308 -308 3
-308 -308 4
257 -257 4
-257 -257 4
-257 -257 4
-257 -257 4
-964 -497 -1
-964 -257 -1
-964 -964 -1
-964 259 1
-964 -964 1
-964 259 -2
-771 -768 1
475 999 -6
475 475 -6
28 -769 3
28 769 3
28 769 3

In [13]:
def partition(array, begin, end):
    pivot = begin
    for i in range(begin+1, end+1):
        if array[i] <= array[begin]:
            pivot += 1
            array[i], array[pivot] = array[pivot], array[i]
    array[pivot], array[begin] = array[begin], array[pivot]
    return pivot



def quicksort(array, begin=0, end=None):
    if end is None:
        end = len(array) - 1
    def _quicksort(array, begin, end):
        if begin >= end:
            return
        pivot = partition(array, begin, end)
        _quicksort(array, begin, pivot-1)
        _quicksort(array, pivot+1, end)
    return _quicksort(array, begin, end)

In [14]:
@given(st.lists(st.integers(), max_size=100))
def test_qsort(lst):
    print(lst)
    expected = sorted(lst)
    quicksort(lst)
    assert lst == expected, (lst, expected)
    
test_qsort()

[]
[0]
[-18087]
[-14536]
[32086]
[-51]
[17094]
[1450374922]
[1450374922, -118]
[-97]
[-43]
[-63]
[-11669, 2614350051549833591, 317865048]
[-11669, 72, 9961, 9630268]
[-11669, 72, 9961, -11669]
[-11669, 72, 9961]
[-11669, -11669, 9961]
[-45, -11669, 9961]
[-45, 9961, 9961]
[15531, -27831, 7782, 118, -21, 58, -7915, -107, 28399, -8610, 16900]
[15531, -27831, 7782, 118, -21, 58, -7915, -107, 28399, -8610, 16900]
[15531, -27831, 7782, 118, 3, 58, -7915, -107, 28399, -8610, 16900]
[15531, -27831, 30, -612721731303885990474787683708371457, -107, 28399, -8610, 16900]
[15531, -27831, 30, -612721731303885990474787683708371457, -107, 28399, -8610, 16900]
[-8610, -27831, 30, -612721731303885990474787683708371457, -107, 28399, -8610, 16900]
[-8610, -27831, 30, -612721731303885990474787683708371457, -8610, 28399, -8610, 16900]
[-22127]
[-22127]
[73, -37746081747451484602111805885716052202, 6838592118124106067, -1265]
[73, -37746081747451484602111805885716052202, 6838592118124106067, -1265, -18, 876

SyntaxError: invalid syntax (Temp/ipykernel_16084/809897477.py, line 4)

# Module unittests

More versatile tool and flexible than plain asserts

In [15]:
import unittest


class TestFrange(unittest.TestCase):

    def test_frange_single_param(self):
        # self.assertEqual(list(frange(5)) , [0, 1, 2, 3, 4])
        self.assertEqual(list(frange(5)), [0, 1, 2, 3, 4, 5])
    
    def test_frange_2_params(self):
        self.assertEqual(list(frange(2, 5)), [2, 3, 4])
        
    def test_frange_2_params_empty_result(self):
        self.assertEqual(list(frange(5, 2)), [])

    def test_frange_custom_step(self):
        self.assertEqual(list(frange(2, 10, 2)), [2, 4, 6, 8])

    def test_frange_reverse_order(self):
        self.assertEqual(list(frange(10, 2, -2)), [10, 8, 6, 4])

unittest.main(argv=[''], verbosity=2, exit=False)

test_frange_2_params (__main__.TestFrange.test_frange_2_params) ... ok
test_frange_2_params_empty_result (__main__.TestFrange.test_frange_2_params_empty_result) ... ok
test_frange_custom_step (__main__.TestFrange.test_frange_custom_step) ... ok
test_frange_reverse_order (__main__.TestFrange.test_frange_reverse_order) ... ok
test_frange_single_param (__main__.TestFrange.test_frange_single_param) ... FAIL

FAIL: test_frange_single_param (__main__.TestFrange.test_frange_single_param)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\mikol\AppData\Local\Temp\ipykernel_31676\451255603.py", line 8, in test_frange_single_param
    self.assertEqual(list(frange(5)), [0, 1, 2, 3, 4, 5])
AssertionError: Lists differ: [Decimal('0'), Decimal('1'), Decimal('2'), Decimal('3'), Decimal('4')] != [0, 1, 2, 3, 4, 5]

Second list contains 1 additional elements.
First extra element 5:
5

- [Decimal('0'), Decimal('1'), Decimal('2'), De

In [ ]:
1. normal cases
2. max and min. 
3. max+1 and min-1
4. wrong types
5. excepts
6. empty value



test1 -> 123
test2 -> 12352


#### Run:

python -m unittests test_frange.py

unittest.main()

In [ ]:
# Names are important: class Test*, test_* for methods

In [16]:
class TestFileOperations(unittest.TestCase):

    def setUp(self):
        print('setUp')
        self._file = open('/tmp/file.txt', 'w+')

    def tearDown(self):
        print('tearDown')
        self._file.close()

    def test_write_operation(self):
        data = 'test'
        self._file.write(data)
        self._file.seek(0)
        data_read = self._file.read()
        self.assertEqual(data_read, data)

    def test_truncate_operation(self):
        data = 'test'
        self._file.write(data)
        self._file.truncate(0)
        data_read = self._file.read()
        self.assertEqual(data_read, '')

    def test_write_on_closed_file(self):
        data = 'test'
        self._file.write(data)
        self._file.close()
        with self.assertRaises(ValueError) as ex:
            self._file.write(data)
        self.assertIsInstance(ex.exception, ValueError)

    ...

In [ ]:
1. Атомарні
2. Швидкі
3. Ізольовані
4. Інкапсульовані
5. Правильні
6. Різнопланові

# PyTest

unittests vs pytest:
1. Less verbose code, more verbose error description
2. Pythonic (PEP-8), doesn't enforce classes
3. Compatible with unittests (recognize TestCase classes and execute them)

pip install pytest

python -m pytest tested_file.py

In [17]:
def test_passing():
    assert (1, 2, 3) == (1, 2, 3)
    
# test is ready :)

In [18]:
import pytest
import ipytest

@pytest.fixture
def file():
    file = open('/tmp/file.txt', 'w+')
    yield file
    file.close()

def test_write_operation(file):
    data = 'test'
    file.write(data)
    file.seek(0)
    data_read = file.read()
    assert data_read == data

def test_truncate_operation(file):
    data = 'test'
    file.write(data)
    file.truncate(0)
    data_read = file.read()
    assert data_read == ''

def test_write_on_closed_file(file):
    data = 'test'
    file.write(data)
    file.close()
    with pytest.raises(ValueError) as ex:
        file.write(data)
    assert isinstance(ex.value, ValueError)

ipytest.run()

======================================= test session starts =======================================
platform win32 -- Python 3.12.2, pytest-8.2.0, pluggy-1.5.0
rootdir: D:\Hillel\python_materials
plugins: anyio-4.3.0, hypothesis-6.100.2
collected 14 items

t_8a6674bb276c4fd7a8234b7f9bb62114.py .

D:\Hillel\python_materials\venv\Lib\site-packages\_hypothesis_pytestplugin.py:111: HypothesisSideeffectWarning: Slow code in plugin: avoid lazy evaluation of (integers(min_value=-1000, max_value=1000),) between importing hypothesis and loading the hypothesis plugin!  Set PYTHONWARNINGS=error to get a traceback and show which plugin is responsible. Additionally, set HYPOTHESIS_EXTEND_INITIALIZATION=1 to pinpoint the exact location.
  notice_initialization_restarted()


.....FFFF.EEE                                         [100%]

============================================= ERRORS ==============================================
_____________________________ ERROR at setup of test_write_operation ______________________________

    @pytest.fixture
    def file():
>       file = open('/tmp/file.txt', 'w+')

C:\Users\mikol\AppData\Local\Temp\ipykernel_31676\2737270264.py:6: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

file = '/tmp/file.txt', args = ('w+',), kwargs = {}

    @functools.wraps(io_open)
    def _modified_open(file, *args, **kwargs):
        if file in {0, 1, 2}:
            raise ValueError(
                f"IPython won't let you open fd={file} by default "
                "as it is likely to crash IPython. If you know what you are doing, "
                "you can use builtins' open."
            )
    
>       return io_open(file, *args, **kwargs)
E       FileNotFoundError: [Errno 

<ExitCode.TESTS_FAILED: 1>

ERROR: usage: ipykernel_launcher.py [options] [file_or_dir] [file_or_dir] [...]
ipykernel_launcher.py: error: unrecognized arguments: -f
  inifile: None
  rootdir: D:\Hillel\python_materials



<ExitCode.USAGE_ERROR: 4>

In [ ]:
1. Для перевірки коду + функціоналу
2. Тести як певний документації
3. Рефакторинг
4. 

# Best practices

1. Cover corner cases: 0, 1, -1, len(lst)-1, len(lst)+1
   - Test symmetry behaviour is also often good idea: a + b == b + a, decode/encode, load/dump, etc
2. Cover all groups (equivalence class) of inputs: positive, negative, empty, large, small, ...
3. Don't repeate tests for the same class: it is redundant.
4. Tests should be isolated from each other (no order dependency)
   - unittest might run tests in arbitrary order
5. Tests should be atomic (test 1 feature), small and fast
6. Tests should cover not only positive (when code works as expected), but negative cases as well (when code doesn't work when it is not supposed)
   - test that errors are returned and exceptions are thrown
7. External dependencies can be replaced with mocks.
   - We don't want to test system libraries or frameworks

In [33]:
# Bad examples

class TestFrange(unittest.TestCase):

    def test_frange_single_param(self):
        self.assertEqual(list(frange(5)), [0, 1, 2, 3, 4])
        self.assertEqual(list(frange(7)), [0, 1, 2, 3, 4, 5, 6])

    def test_frange_2_params(self):
        self.assertEqual(list(frange(2, 5)), [0, 1, 2, 3, 4, 5])
        self.assertEqual(list(frange(5, 2)), [])
    

In [24]:
1. Перевірити код
2. Рефакторинг
3. Документація
4. Економія

SyntaxError: invalid syntax (4163957638.py, line 1)

In [ ]:
# TDD

